# mesos的framework  

我学习的是**mesos-0.13.0**版本。  
从 **src/examplels/test_framework.cpp** 入手，开始看。记录的是我自己的理解，有些地方可能理解的不到位，写的有问题，仅供参考。  
test_famework.cpp 中包含了 `TestFramework:Scheduler类`，和一个 `main函数` 。  

这一部分主要内容是：  
* TestScheduler:Scheduler  
* TestSchedulerDriver:SchedulerDriver  
* 还有就是libprocess中的内容（需要用另一个篇幅记录）

---

## Mesos与Framework的关系：  
![mesos-architecture.png](./pictures/mesos-architecture.png)

**我的理解：**  
如Mesos基本架构图所示，Framework位于Mesos的上层，作为Mesos的应用层，向Mesos提交计算任务。  

### Mesos上运行的Framework的组成：  

在 Mesos 上运行的 framework 由两部分组成：  
1. 一个是 `scheduler` ，通过注册到 master 来**获取集群资源**。  
2. 另一个是在 slave 节点上运行的 `executor` 进程，它为 framework 的 task 提供了运行环境。(后续会用另一篇记录)  

Master 决定为每个 framework 提供多少资源（ Mesos 一开始采用最大最小分配策略，之后使用 RDF 分配策略）， framework 的 scheduler 来选择其中提供的资源。当 framework 同意了提供的资源，它通过 master 将 task 发送到提供资源的slaves 上运行。
> * **疑问：** 这里framework是如何选择资源的呢？  
> * **注明：** TestScheduler中的 resourceOffer(...) 方法是在 framework 向master 注册后，收到 master 的 resourceOffer信息 后被调用的。 
> * **我的理解：** TestScheduler 中实现的 resourceOffer方法 的第二个参数为 Offer类型 的数组，每一个 Offer 对应的是一个 slave 机器的资源，集群的全部资源抽象为一个资源数组。要使用这些资源，首先**要实现一个在资源数组中查找合适自己资源需求的模块**，这里涉及到了 framework 如何挑选自己合适的资源这么一说。具体的 `Spark` 这一部分的代码还需要进一步的研究（还没找到在哪里），而 TestFramework 的 resourceOffer 函数内容比较简单。其次，调用 lanchTask方法 直接使用该资源。
> > **在master提供resourceoffer之前**，master会把集群里面所有的资源都映射为一个个Offer，把这些资源暂时都标为已被占用的，然后发给一个framework，这个framework选中他想要的资源之后，构造在launchtasksmessage里面发送给master，master再把这些framework真正选中的资源标记为已用，其他的资源又重新标记成空闲的，然后再发给下一个framework
> > （上面这一部分应该会在master模块看到具体的代码-待定）。

> * **疑问：** 具体 framework 如何使用资源， test_framework.cpp 中的resourceOffer()方法 简单的写了一个测试的 task数组 当作是计算任务了。是吗？另外这里的taskInfo是干啥的？  
> * **注明：** 这里的 SchedulerDriver 的 launchTask 方法也是一个深入的内容。

```C++
 virtual void resourceOffers(SchedulerDriver* driver, const vector<Offer>& offers)

  {
      //确认提供的资源信息
      //  ...
      
      // Launch tasks.
      vector<TaskInfo> tasks;
      while (tasksLaunched < totalTasks &&
             cpus >= CPUS_PER_TASK &&
             mem >= MEM_PER_TASK) {
        int taskId = tasksLaunched++;
        cout << "Starting task " << taskId << " on " << offer.hostname() << endl;
        TaskInfo task;

        task.set_name("Task " + lexical_cast<string>(taskId));
        task.mutable_task_id()->set_value(lexical_cast<string>(taskId));
        task.mutable_slave_id()->MergeFrom(offer.slave_id());
        task.mutable_executor()->MergeFrom(executor);
       
        Resource* resource;
        resource = task.add_resources();
        resource->set_name("cpus");
        resource->set_type(Value::SCALAR);
        resource->mutable_scalar()->set_value(CPUS_PER_TASK);
        resource = task.add_resources();
        resource->set_name("mem");
        resource->set_type(Value::SCALAR);
        resource->mutable_scalar()->set_value(MEM_PER_TASK);
        tasks.push_back(task);
        cpus -= CPUS_PER_TASK;
        mem -= MEM_PER_TASK;
      }
      driver->launchTasks(offer.id(), tasks);
    }
  }
```
（下面一小段来自其网友们，不太理解。我的理解下面的意思是，mesos 也可以直接执行 shell 脚本等短周期的、独立起线程的小计算任务。）
事实上，Framework 不仅可以是通用的框架，也可以是像 Hadoop 的 Job 或者 YARN 的 Application 那样的简单计算任务，也就是说，Framework并非必须是一个`framework`，或者一个长时间运行的服务（比如JobTracker等），也可以是一个短生命周期的Job或者Application。

## test_framework.cpp的main函数  
```C++
int main(int argc, char** argv)
{
//环境变量等准备代码...

//初始化ExecutorInfo,TestScheduler,FrameworkInfo,MesoSchedulerDriver
  ExecutorInfo executor;
  //...
  TestScheduler scheduler(executor);
  FrameworkInfo framework;
  //...
  MesosSchedulerDriver driver(&scheduler, framework, argv[1]);
  return driver.run() == DRIVER_STOPPED ? 0 : 1;
}
```  
`main()函数`内容很短，其中只实例化了4个类，主要作用就是启动了framework的process进程，负责监听信号的：  
1. **ExecutorInfo** : protobuf定义的 ExecutorInfo 类，是一个Message。 
2. **TestScheduler** : 在 test_framework.cpp中声明，继承了`Scheduler`类(scheduler.hpp文件中声明)，像前面说的那样**用于获取集群资源**。TestScheduler 对象要将 ExecutorInfo类 的对象 executor 作为参数传入，之后初始化 TestScheduler类 的对象 scheduler 。  
3. **FrameworkInfo** : （同1）  
4. **MesosSchedulerDriver** : 在 `scheduler.hpp` 中声明，在 `sched.cpp` 中实现，继承 `SchedulerDriver`类 。调用 driver 的 run 方法，目的是 spawn 一个 process（可以理解为通信代理），即 MesosSchedulerDriver 类的私有变量- SchedulerProcess 的对象 process（即启动一个process进程，监听外界给 MesosSchedulerDriver 发送的信息）。  
> **Scheduler类** 用于获取集群资源，到其中声明了多个虚函数来实现获取集群资源的功能，具体的 **TestScheduler类** 实现了 framework 向 master 注册、到收到resourceoffe等情况的回调函数，其中还有一个私有成员变量ExecutorInfo。
> > **TestScheduler类** 中 framework 的 registered , reregistered , disconnected 操作都没有写具体的操作，留了一个空函数。`resourceOffers` 函数实现了a)资源的确认，b)接受资源，c)使用资源三个部分。  函数的第二个参数是 `Offer类型` 的vector数组，最后调用 SchedulerDriver的 lanchtask方法 执行任务（直接使用资源）。
> > 
> > **我感觉**，一个`framework` 与一个 `scheduler` 一一对应，在 mesos 代码中， scheduler 的对象可以代表一个 framework 。并且，在 TestScheduler类 的具体方法中， SchedulerDriver 对象作为参数传进去，然后被调用这个 SchedulerDriver对象 的方法。所以这里可以看出 SchedulerDriver 像是一个 Scheduler 的**外层通信代理**，当 Scheduler 有操作要执行时，执行对应的SchedulerDriver里的方法与外界进行通信，同样的， SchedulerDriver 接收到外部消息时，再调用 Scheduler 的相应方法。（我的理解）**  
> > 
> > **疑问：** framework 在哪里发送自己接受本次 resourceoffer 的资源信号？  
> >               不清楚 ExecutorInfo 中参数的具体含义，其中 Resource 这个 Message 的使用方法也不懂。


```C++
    class Scheduler
    {
    public:
      virtual ~Scheduler() {}

    //framework向master注册信息时，函数被唤醒
      virtual void registered(SchedulerDriver* driver,
                              const FrameworkID& frameworkId,
                              const MasterInfo& masterInfo) = 0;

    //当新的Mesos master被选举出来后，当前的frameword的scheduler要向新的master重新注册，函数被唤醒
      virtual void reregistered(SchedulerDriver* driver,
                                const MasterInfo& masterInfo) = 0;

    //当framework的scheduler与master失联时，函数被唤醒。
      virtual void disconnected(SchedulerDriver* driver) = 0;

    //当framework收到master的resourceoffer信息时，函数被唤醒。
    //framework收到的resourceoffer信息是一个Offer类型的数组，每一个Offer值即一个计算节点的资源信息。
    //如果这个framework已经拒绝了这个offer或者是这个offer已经被rescind了，那master就不会再给这个framework发这个offer了。
      virtual void resourceOffers(SchedulerDriver* driver,
                                  const std::vector<Offer>& offers) = 0;

    //当一份resourceoffer不再可用时（slave机器丢失，其他framework已经使用这个resourceoffer中的资源），函数被唤醒。
      virtual void offerRescinded(SchedulerDriver* driver,
                                  const OfferID& offerId) = 0;

       //当task状态改变时（slave丢失、task丢失时，executor把任务完成后发送的任务完成信息等情况），被唤醒。
      virtual void statusUpdate(SchedulerDriver* driver,
                                const TaskStatus& status) = 0;

       //当executor发送消息时？？？，函数被唤醒。
      virtual void frameworkMessage(SchedulerDriver* driver,
                                    const ExecutorID& executorId,
                                    const SlaveID& slaveId,
                                    const std::string& data) = 0;

       //slave被确认丢失时，函数被唤醒。（framework收到这个消息后，需要重新运行分配在丢失slave机器上的task）
      virtual void slaveLost(SchedulerDriver* driver,
                             const SlaveID& slaveId) = 0;

       //当executor正常退出或异常终止时，函数被唤醒。
      virtual void executorLost(SchedulerDriver* driver,
                                const ExecutorID& executorId,
                                const SlaveID& slaveId,
                                int status) = 0;

       //scheduler/scheduler driver出现其他异常时，唤醒函数。
      virtual void error(SchedulerDriver* driver, const std::string& message) = 0;
    };  
```  

> **SchedulerDriver类** 是一个将`scheduler`与Mesos连接起来的抽象接口。 此接口用于管理scheduler的生命周期（启动它，停止它或等待它完成）以及与Mesos交互（例如，启动任务，终止任务等）。**MesosSchedulerDriver类** 继承了 SchedulerDiriver类 。
> > * **MesosSchedulerDriver类：** 在 `shceduler.hpp` 文件中声明，在 `sched.cpp` 文件中定义具体的方法。私有变量中的 `SchedulerProcess类` 的对象 `process` 负责与外界通信的部分。 MesosSchedulerDriver类的构造函数中调用了 process::initialize() ，这个方法在 process.hpp 中定义，并在 process.cpp 中实现，默认参数为空字符串。
> > **我的想法：** MesosSchedulerDriver这一块的具体实现需要继续深入看（另一个篇幅描述）.

```C++
  //MesosSchedulerDriver类的私有变量
  Scheduler* scheduler;
  FrameworkInfo framework;
  std::string master;

  // Libprocess process for communicating with master.
  internal::SchedulerProcess* process;

  // Mutex to enforce all non-callbacks are execute serially.
  pthread_mutex_t mutex;

  // Condition variable for waiting until driver terminates.
  pthread_cond_t cond;

  // Current status of the driver.
  Status status;

```


![mesos-process.png](./pictures/mesos-process.png)  

(图片来自网络)